# Importando bibliotecas

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window as W
from pyspark.sql.types import *

# Criando Spark session

In [2]:
spark = (SparkSession
        .builder
        .master('local')
        .appName('Pyspark_MG')
        .config('spark.ui.port', '4050')
        .getOrCreate()
        )

# Extração de dados - Minas Gerais

In [3]:
# Leitura do arquivo do bucket em formato parquet
df = spark.read.parquet('gs://projeto_final_2021/pandas_to_parquet/MG_parquet/df_optimized.snappy')

In [4]:
# Visualização do schema e tipo de dados
df.printSchema()

root
 |-- masp: double (nullable = true)
 |-- nome: string (nullable = true)
 |-- descsitser: string (nullable = true)
 |-- nmefet: string (nullable = true)
 |-- tem_apost: string (nullable = true)
 |-- desccomi: string (nullable = true)
 |-- descinst: string (nullable = true)
 |-- descunid: string (nullable = true)
 |-- carga_hora: double (nullable = true)
 |-- remuner: double (nullable = true)
 |-- teto: double (nullable = true)
 |-- judic: string (nullable = true)
 |-- ferias: double (nullable = true)
 |-- decter: double (nullable = true)
 |-- premio: long (nullable = true)
 |-- feriasprem: double (nullable = true)
 |-- jetons: double (nullable = true)
 |-- eventual: double (nullable = true)
 |-- ir: double (nullable = true)
 |-- prev: double (nullable = true)
 |-- rem_pos: double (nullable = true)
 |-- bdmg: double (nullable = true)
 |-- cemig: double (nullable = true)
 |-- codemig: long (nullable = true)
 |-- cohab: double (nullable = true)
 |-- copasa: double (nullable = true)
 |

## Transformações

In [15]:
# Selecionando colunas para manter no data frame
cols_to_keep = {'mes', 'ano', 'nome', 'descsitser', 'nmefet', 'remuner', 'rem_pos'}
# Selecionando colunas para exluir do data frame 
cols_to_drop = list(set(df.schema.names).difference(cols_to_keep))
# Definido valor padrão para coluna estado
value = 'MG'
# Realizando drop das colunas não desejáveis e adicionando a coluna estado
df_mg = df.drop(*cols_to_drop).withColumn('estado', lit(value).cast('string'))

In [16]:
# Renomeando colunas para melhor identificação
df_mg = (df_mg.withColumnRenamed('descsitser','situacao')
              .withColumnRenamed('nmefet','cargo')
              .withColumnRenamed('remuner','salarioBruto')
              .withColumnRenamed('rem_pos','salarioLiquido'))

# Gerando visualização
df_mg.show()

+--------------------+--------+--------------------+------------+--------------+----+---+------+
|                nome|situacao|               cargo|salarioBruto|salarioLiquido| ano|mes|estado|
+--------------------+--------+--------------------+------------+--------------+----+---+------+
|   AARAO LOPES COSTA|   ATIVO|                CABO|      5692.0|       4668.11|2020|  1|    MG|
|AARAO MARQUES DA ...|   ATIVO|AGENTE DE SEGURAN...|      4778.5|       4117.18|2020|  1|    MG|
|  AARON DUARTE DALLA|   ATIVO|ESPEC.EM POLITICA...|     20490.7|      16217.44|2020|  1|    MG|
|AARON FERRAREZ BO...|   ATIVO|             SD 1 CL|     4508.26|       3728.25|2020|  1|    MG|
|AARON FRANCA TEOFILO|   ATIVO|PROFESSOR DE EDUC...|     3124.36|       3875.45|2020|  1|    MG|
|ABADIA APARECIDA ...|   ATIVO|AUXILIAR DE SERVI...|     1091.13|       1338.45|2020|  1|    MG|
|ABADIA APARECIDA ...|   ATIVO|AUXILIAR DE SERVI...|     1128.76|       1384.61|2020|  1|    MG|
|ABADIA APARECIDA ...|   ATIVO

# Salvar arquivo

In [14]:
# Salvando em formato parquet
(df_mg.write.format("parquet")
.option("header", "true")
.option("inferschema", "true")
.save("gs://projeto_final_2021/parquet_to_bq/MG/")
)